In [ ]:
#Regular HMA

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    cwd = Path.cwd()
    for folder in (cwd, *cwd.parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r'\s+', header=None)
            break
    else:
        raise FileNotFoundError(f"'{prices_file}' not found")
    if not (0 <= instrument < df.shape[1]):
        raise IndexError(f"Instrument must be in [0, {df.shape[1]-1}]")
    return df.iloc[:, instrument].values

def wma(prices: np.ndarray, period: int) -> np.ndarray:
    n = len(prices)
    out = np.full(n, np.nan)
    w = np.arange(1, period+1)
    S = w.sum()
    for i in range(period-1, n):
        window = prices[i-period+1:i+1]
        out[i] = (w * window).sum() / S
    out[:period-1] = prices[:period-1]
    return out

def hma(prices: np.ndarray, period: int) -> np.ndarray:
    half  = max(1, period//2)
    sqrtp = max(1, int(np.sqrt(period)))
    h1 = wma(prices, half)
    h2 = wma(prices, period)
    diff = 2*h1 - h2
    return wma(diff, sqrtp)

def plot_hma_signs_with_dividers(
    instrument: int,
    prices_file: str = 'prices.txt',
    period: int = 11,
    T1: int = 200,
    T2: int = 400
):
    prices = load_prices(prices_file, instrument)
    n = len(prices)
    T2 = min(T2, n)

    # compute HMA and normalized gradient
    smooth = hma(prices, period)
    grad   = np.zeros(n)
    grad[1:] = (smooth[1:] - smooth[:-1]) / smooth[:-1]

    # binary sign: +1 if gradient>0 else -1
    signs = np.where(grad > 0, 1, -1)

    # slice
    x         = np.arange(T1, T2)
    p_slice   = prices[T1:T2]
    s_slice   = smooth[T1:T2]
    sign_slice= signs[T1:T2]

    # plot
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p_slice, label='Raw Price', color='black')
    ax.plot(x, s_slice, '--', label=f'HMA({period})', color='orange', lw=2)

    # draw thin black dividers at each timestep boundary
    for xi in x:
        ax.axvline(xi, color='black', linewidth=0.3, alpha=0.5, zorder=0)

    y0, y1 = p_slice.min(), p_slice.max()
    # shade contiguous runs cleanly
    start = x[0]
    curr  = sign_slice[0]
    for xi, si in zip(x, sign_slice):
        if si != curr:
            color = 'green' if curr==1 else 'red'
            ax.axvspan(start, xi, ymin=0, ymax=1,
                       facecolor=color, alpha=0.2, edgecolor=None)
            start, curr = xi, si
    # last run
    color = 'green' if curr==1 else 'red'
    ax.axvspan(start, x[-1]+1, ymin=0, ymax=1,
               facecolor=color, alpha=0.2, edgecolor=None)

    ax.set_title(f'Instrument {instrument}: Price + HMA({period}) Signals (t={T1}-{T2})')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Price')
    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    plot_hma_signs_with_dividers(
        instrument=1,
        prices_file='prices.txt',
        period=4,
        T1=300,
        T2=400
    )


In [ ]:
#HMA with larger window

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    cwd = Path.cwd()
    for folder in (cwd, *cwd.parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r'\s+', header=None)
            break
    else:
        raise FileNotFoundError(f"'{prices_file}' not found")
    if not (0 <= instrument < df.shape[1]):
        raise IndexError(f"Instrument must be in [0, {df.shape[1]-1}]")
    return df.iloc[:, instrument].values

def wma(prices: np.ndarray, period: int) -> np.ndarray:
    n = len(prices)
    out = np.full(n, np.nan)
    w = np.arange(1, period+1)
    S = w.sum()
    for i in range(period-1, n):
        window = prices[i-period+1:i+1]
        out[i] = (w * window).sum() / S
    out[:period-1] = prices[:period-1]
    return out

def hma(prices: np.ndarray, period: int) -> np.ndarray:
    half  = max(1, period//2)
    sqrtp = max(1, int(np.sqrt(period)))
    h1 = wma(prices, half)
    h2 = wma(prices, period)
    diff = 2*h1 - h2
    return wma(diff, sqrtp)

def plot_hma_signs_with_dividers(
    instrument: int,
    prices_file: str = 'prices.txt',
    period: int = 11,
    T1: int = 200,
    T2: int = 400
):
    prices = load_prices(prices_file, instrument)
    n = len(prices)
    T2 = min(T2, n)

    # compute HMA and normalized gradient
    smooth = hma(prices, period)
    grad   = np.zeros(n)
    grad[1:] = (smooth[1:] - smooth[:-1]) / smooth[:-1]

    # binary sign: +1 if gradient>0 else -1
    signs = np.where(grad > 0, 1, -1)

    # slice
    x         = np.arange(T1, T2)
    p_slice   = prices[T1:T2]
    s_slice   = smooth[T1:T2]
    sign_slice= signs[T1:T2]

    # plot
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p_slice, label='Raw Price', color='black')
    ax.plot(x, s_slice, '--', label=f'HMA({period})', color='orange', lw=2)

    # draw thin black dividers at each timestep boundary
    for xi in x:
        ax.axvline(xi, color='black', linewidth=0.3, alpha=0.5, zorder=0)

    y0, y1 = p_slice.min(), p_slice.max()
    # shade contiguous runs cleanly
    start = x[0]
    curr  = sign_slice[0]
    for xi, si in zip(x, sign_slice):
        if si != curr:
            color = 'green' if curr==1 else 'red'
            ax.axvspan(start, xi, ymin=0, ymax=1,
                       facecolor=color, alpha=0.2, edgecolor=None)
            start, curr = xi, si
    # last run
    color = 'green' if curr==1 else 'red'
    ax.axvspan(start, x[-1]+1, ymin=0, ymax=1,
               facecolor=color, alpha=0.2, edgecolor=None)

    ax.set_title(f'Instrument {instrument}: Price + HMA({period}) Signals (t={T1}-{T2})')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Price')
    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    plot_hma_signs_with_dividers(
        instrument=1,
        prices_file='prices.txt',
        period=16,
        T1=300,
        T2=430
    )


In [ ]:
#Kalman filtering

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ── Kalman smoother (unchanged) ───────────────────────────────────────────────
def kalman_trend_smoother(prices, R=1.0, Q_level=1e-3, Q_trend=1e-5):
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1],
                    [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)

    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y   = prices[t] - (H @ x_prior)[0]
        S   = (H @ P_prior @ H.T)[0, 0] + R
        K   = (P_prior @ H.T) / S
        x   = x_prior + (K.flatten() * y)
        P   = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ── Plot with binary long/short and neutral threshold ─────────────────────────
def plot_kalman_binary(
    instrument       : int,
    prices_file      : str   = "prices.txt",
    R                : float = 0.05,
    Q_level          : float = 4e-3,
    Q_trend          : float = 1e-5,
    T1               : int   = 300,
    T2               : int   = 350,
    pct_thresh       : float = 0.001,      # 0.1 % neutral band on price move
    divider_width    : float = 0.3,
    divider_alpha    : float = 0.45,
):
    # ── load prices ───────────────────────────────────────────────────────────
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    prices = df.iloc[:, instrument].values

    # ── Kalman smoother + gradient & price change ─────────────────────────────
    smooth = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    grad   = np.zeros_like(smooth)
    grad[1:] = (smooth[1:] - smooth[:-1]) / smooth[:-1]

    pct_change = np.zeros_like(prices)
    pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]

    # ── Build signal:  +1 long, -1 short, 0 neutral when |pct_change| < thresh
    sign = np.where(np.abs(pct_change) < pct_thresh,            0,
             np.where(grad > 0,                                1, -1))

    # ── slice window ──────────────────────────────────────────────────────────
    T2 = min(T2, len(prices))
    x           = np.arange(T1, T2)
    p_window    = prices[T1:T2]
    s_window    = smooth[T1:T2]
    sign_window = sign[T1:T2]

    # ── plotting ──────────────────────────────────────────────────────────────
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p_window, color="black", label="Raw Price")
    ax.plot(x, s_window, "--", color="orange", lw=2, label="Kalman Smoothed")

    # thin black dividers at each step
    for xi in x:
        ax.axvline(xi, color="black",
                   lw=divider_width, alpha=divider_alpha, zorder=0)

    # shade contiguous long / short blocks; leave neutral bars white
    run_start, curr = x[0], sign_window[0]
    for xi, si in zip(x, sign_window):
        if si != curr:
            if curr == 1 or curr == -1:           # skip neutral (0) shading
                ax.axvspan(run_start, xi,
                           facecolor="green" if curr == 1 else "red",
                           alpha=0.22, edgecolor=None)
            run_start, curr = xi, si
    # last run
    if curr == 1 or curr == -1:
        ax.axvspan(run_start, x[-1] + 1,
                   facecolor="green" if curr == 1 else "red",
                   alpha=0.22, edgecolor=None)

    ax.set_title(f"Instrument {instrument}: Price & Kalman Signals "
                 f"(t={T1}-{T2}, neutral |ΔP|<{pct_thresh*100:.2f}%)")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

# ── Example ───────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    plot_kalman_binary(
        instrument   = 1,
        prices_file  = "prices.txt",
        R            = 0.075,
        Q_level      = 4e-3,
        Q_trend      = 1e-5,
        T1           = 300,
        T2           = 450,
        pct_thresh   = 0.000     # 0.1 % neutral band
    )


In [ ]:
#Combination of HMA and Kalman smoothing, based on agreement and netrality

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─────────────────────────────────────────────────────────────────────────
# Helpers
# ─────────────────────────────────────────────────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values


def wma(prices: np.ndarray, period: int) -> np.ndarray:
    n = len(prices)
    out = np.full(n, np.nan)
    w   = np.arange(1, period + 1)
    S   = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * prices[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = prices[: period - 1]
    return out


def hma(prices: np.ndarray, period: int) -> np.ndarray:
    half  = max(1, period // 2)
    sqrtp = max(1, int(np.sqrt(period)))
    return wma(2 * wma(prices, half) - wma(prices, period), sqrtp)


def kalman_trend_smoother(prices, R=1.0, Q_level=1e-3, Q_trend=1e-5):
    n = len(prices)
    x = np.array([prices[0], 0.0])
    P = np.eye(2)
    F = np.array([[1, 1], [0, 1]])
    H = np.array([[1, 0]])
    Q = np.diag([Q_level, Q_trend])

    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out


# ─────────────────────────────────────────────────────────────────────────
# Plot – colour only where *both* HMA & Kalman signals agree
# ─────────────────────────────────────────────────────────────────────────
def plot_combined_signals(
    instrument: int,
    prices_file: str = "prices.txt",
    hma_period: int = 11,
    # Kalman params
    R: float = 0.05,
    Q_level: float = 4e-3,
    Q_trend: float = 1e-5,
    # neutral band for Kalman price change
    pct_thresh: float = 0.001,
    # window
    T1: int = 300,
    T2: int = 400,
    divider_width: float = 0.3,
    divider_alpha: float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n = len(prices)
    T2 = min(T2, n)

    # ---- HMA section -------------------------------------------------------
    hma_series = hma(prices, hma_period)
    hma_grad   = np.zeros(n)
    hma_grad[1:] = (hma_series[1:] - hma_series[:-1]) / hma_series[:-1]
    hma_sign   = np.where(hma_grad > 0, 1, -1)

    # ---- Kalman section ----------------------------------------------------
    kalman_series = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    kal_grad = np.zeros(n)
    kal_grad[1:] = (kalman_series[1:] - kalman_series[:-1]) / kalman_series[:-1]
    pct_change    = np.zeros(n)
    pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]

    kal_sign = np.where(
        np.abs(pct_change) < pct_thresh, 0, np.where(kal_grad > 0, 1, -1)
    )

    # ---- Combined agreement -----------------------------------------------
    combo_sign = np.where((hma_sign == kal_sign) & (kal_sign != 0), hma_sign, 0)

    # ---- Slice window ------------------------------------------------------
    x       = np.arange(T1, T2)
    p_win   = prices[T1:T2]
    h_win   = hma_series[T1:T2]
    k_win   = kalman_series[T1:T2]
    c_win   = combo_sign[T1:T2]

    # ---- Plot --------------------------------------------------------------
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p_win, label="Raw Price", color="black")
    ax.plot(x, h_win, "--", label=f"HMA({hma_period})", color="C1")
    ax.plot(x, k_win, "-.", label="Kalman Smoothed", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_width, alpha=divider_alpha, zorder=0)

    # contiguous shading where both agree
    y0, y1 = p_win.min(), p_win.max()
    start, curr = x[0], c_win[0]
    for xi, sig in zip(x, c_win):
        if sig != curr:
            if curr != 0:
                ax.axvspan(
                    start,
                    xi,
                    facecolor="green" if curr == 1 else "red",
                    alpha=0.22,
                    edgecolor=None,
                )
            start, curr = xi, sig
    # last span
    if curr != 0:
        ax.axvspan(
            start,
            x[-1] + 1,
            facecolor="green" if curr == 1 else "red",
            alpha=0.22,
            edgecolor=None,
        )

    ax.set_title(
        f"Instrument {instrument}: Price with AGREED Signals (t={T1}-{T2})\n"
        f"Green/Red only when HMA & Kalman agree; neutral elsewhere"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


# ───────────────────────── Example ───────────────────────────────────────────
if __name__ == "__main__":
    plot_combined_signals(
        instrument=1,
        prices_file="prices.txt",
        hma_period=4,
        R=0.075,
        Q_level=4e-3,
        Q_trend=1e-5,
        pct_thresh=0.002,
        T1=300,
        T2=400,
    )


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ── helpers ───────────────────────────────────────────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values


def wma(prices: np.ndarray, period: int) -> np.ndarray:
    n, w = len(prices), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * prices[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = prices[: period - 1]
    return out


def hma(prices: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(prices, half) - wma(prices, period), sqrtp)


# ── buffered regime logic ─────────────────────────────────────────────────────
def buffered_signals(raw_sign: np.ndarray, N: int = 6, X: int = 3) -> np.ndarray:
    """
    Convert a raw ±1 series into a buffered series:

    • If the current regime has already persisted for ≥N bars, flip on the
      first opposite raw signal.
    • Otherwise require X consecutive opposite raw signals before flipping.

    Returns the filtered (+1 / -1) signal series.
    """
    out   = np.empty_like(raw_sign)
    state = raw_sign[0]
    streak_same = 1                 # how long we've been in current state
    opp_count   = 0                 # consecutive opposite raw signs

    out[0] = state
    for t in range(1, len(raw_sign)):
        r = raw_sign[t]

        if r == state:
            # still in same regime
            streak_same += 1
            opp_count = 0
            out[t] = state
        else:
            # opposite raw signal
            opp_count += 1
            if streak_same >= N:
                # strong trend → switch immediately
                state = r
                streak_same = 1
                opp_count   = 0
            elif opp_count >= X:
                # choppy regime → switch only after X confirmations
                state = r
                streak_same = 1
                opp_count   = 0
            else:
                # hold old state
                streak_same = 0      # we reset because raw differs
            out[t] = state
    return out


# ── plot function ─────────────────────────────────────────────────────────────
def plot_hma_buffered(
    instrument  : int,
    prices_file : str  = "prices.txt",
    period      : int  = 8,
    N_trend     : int  = 6,      # ≥N consecutive bars = established trend
    X_confirm   : int  = 3,      # need X confirmations if not established
    T1          : int  = 300,
    T2          : int  = 400,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # HMA + raw slope sign
    smooth   = hma(prices, period)
    grad     = np.zeros(n)
    grad[1:] = (smooth[1:] - smooth[:-1]) / smooth[:-1]
    raw_sign = np.where(grad > 0, 1, -1)          # ±1 each bar

    # buffered logic
    sig = buffered_signals(raw_sign, N_trend, X_confirm)

    # slice
    x   = np.arange(T1, T2)
    p   = prices[T1:T2]
    s   = smooth[T1:T2]
    sg  = sig[T1:T2]

    # plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, s, "--", label=f"HMA({period})", color="orange", lw=2)

    # background shading by contiguous buffered signals
    y0, y1   = p.min(), p.max()
    start, cur = x[0], sg[0]
    for xi, si in zip(x, sg):
        if si != cur:
            ax.axvspan(start, xi,
                       facecolor="green" if cur == 1 else "red",
                       alpha=0.22, edgecolor=None)
            start, cur = xi, si
    ax.axvspan(start, x[-1] + 1,
               facecolor="green" if cur == 1 else "red",
               alpha=0.22, edgecolor=None)

    ax.set_title(
        f"Instr {instrument}: HMA({period}) buffered signals\n"
        f"trend≥{N_trend} flips immediately, otherwise need {X_confirm} confirms"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


# ── demo ──────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    plot_hma_buffered(
        instrument = 1,
        prices_file= "prices.txt",
        period     = 4,
        N_trend    = 5,
        X_confirm  = 1,
        T1         = 300,
        T2         = 400,
    )


In [ ]:
#HMA buffered with Kalman agreement

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────── core helpers ──────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values


def wma(series: np.ndarray, period: int) -> np.ndarray:
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out


def hma(series: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)


def kalman_trend_smoother(prices, R=1.0, Q_level=1e-3, Q_trend=1e-5):
    n = len(prices)
    x = np.array([prices[0], 0.0])
    P = np.eye(2)
    F = np.array([[1, 1], [0, 1]])
    H = np.array([[1, 0]])
    Q = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y   = prices[t] - (H @ x_prior)[0]
        S   = (H @ P_prior @ H.T)[0, 0] + R
        K   = (P_prior @ H.T) / S
        x   = x_prior + (K.flatten() * y)
        P   = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ───────────── buffered HMA logic (trend vs. confirmation) ──────────────────
def buffered_signals(raw_sign: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Return a buffered ±1 series.
    If the current regime has lasted ≥N bars, flip on first opposite tick.
    Otherwise require X opposite ticks before flipping.
    """
    out          = np.empty_like(raw_sign)
    state        = raw_sign[0]
    streak_same  = 1
    opp_count    = 0
    out[0]       = state

    for t in range(1, len(raw_sign)):
        r = raw_sign[t]
        if r == state:
            streak_same += 1
            opp_count = 0
            out[t] = state
        else:                              # opposite raw tick
            opp_count += 1
            if streak_same >= N or opp_count >= X:
                state       = r
                streak_same = 1
                opp_count   = 0
            else:
                streak_same = 0
            out[t] = state
    return out


# ─────────────────────────── combined plot ────────────────────────────────
def plot_hma_kalman_agree(
    instrument   : int,
    prices_file  : str  = "prices.txt",
    # HMA params
    hma_period   : int  = 8,
    N_trend      : int  = 6,
    X_confirm    : int  = 3,
    # Kalman params
    R            : float = 0.05,
    Q_level      : float = 4e-3,
    Q_trend      : float = 1e-5,
    pct_thresh   : float = 0.001,
    # window
    T1           : int   = 300,
    T2           : int   = 400,
    divider_w    : float = 0.3,
    divider_a    : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # ---------- HMA buffered signal ---------------------------------------
    hma_series = hma(prices, hma_period)
    h_grad     = np.zeros(n)
    h_grad[1:] = (hma_series[1:] - hma_series[:-1]) / hma_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_buf      = buffered_signals(h_raw, N_trend, X_confirm)

    # ---------- Kalman signal ---------------------------------------------
    k_series = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    k_grad   = np.zeros(n)
    k_grad[1:] = (k_series[1:] - k_series[:-1]) / k_series[:-1]

    pct = np.zeros(n)
    pct[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_sign  = np.where(np.abs(pct) < pct_thresh, 0,
               np.where(k_grad > 0, 1, -1))

    # ---------- Combined agreement ----------------------------------------
    agree = np.where((h_buf == k_sign) & (k_sign != 0), h_buf, 0)

    # ---------- Slice window ----------------------------------------------
    x  = np.arange(T1, T2)
    pw = prices[T1:T2]
    hw = hma_series[T1:T2]
    kw = k_series[T1:T2]
    aw = agree[T1:T2]

    # ---------- Plot -------------------------------------------------------
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, pw, label="Raw Price", color="black")
    ax.plot(x, hw, "--", label=f"HMA buffered ({hma_period})", color="C1")
    ax.plot(x, kw, "-.", label="Kalman", color="C2")

    # thin dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # contiguous shading where both agree
    y0, y1 = pw.min(), pw.max()
    start, cur = x[0], aw[0]
    for xi, si in zip(x, aw):
        if si != cur:
            if cur != 0:
                ax.axvspan(start, xi,
                           facecolor="green" if cur == 1 else "red",
                           alpha=0.22, edgecolor=None)
            start, cur = xi, si
    if cur != 0:
        ax.axvspan(start, x[-1] + 1,
                   facecolor="green" if cur == 1 else "red",
                   alpha=0.22, edgecolor=None)

    ax.set_title(
        f"Instr {instrument}: AGREED zones (green=both long, red=both short)\n"
        f"HMA buffered N={N_trend}, X={X_confirm}  |  Kalman neutral band {pct_thresh*100:.2f}%"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="lower left")
    plt.tight_layout()
    plt.show()


# ─────────────────────────── demo ────────────────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree(
        instrument  =18,
        prices_file = "prices.txt",
        hma_period  = 4,
        N_trend     = 5,
        X_confirm   = 1,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        pct_thresh  = 0.001,
        T1          = 340,
        T2          = 800
        ,
    )


In [ ]:
#trying so smooth in real time

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ----------------------------------------------------------------------
# helpers (unchanged wma/hma/kalman)   … keep the same implementations …
# ----------------------------------------------------------------------
def load_prices(prices_file: str, inst: int) -> np.ndarray:
    for fld in (Path.cwd(), *Path.cwd().parents):
        f = fld / prices_file
        if f.exists():
            data = pd.read_csv(f, sep=r'\s+', header=None); break
    else:
        raise FileNotFoundError(prices_file)
    if not 0 <= inst < data.shape[1]:
        raise IndexError("bad instrument")
    return data.iloc[:, inst].values

def wma(x, n):
    w = np.arange(1, n+1); S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    return wma(2*wma(x, max(1, n//2)) - wma(x, n), max(1, int(np.sqrt(n))))

def kalman(x, R=0.05, Ql=4e-3, Qt=1e-5):
    n=len(x); s=np.zeros(n); P=np.eye(2); F=np.array([[1,1],[0,1]])
    H=np.array([[1,0]]); Q=np.diag([Ql,Qt]); z=x
    state=np.array([z[0],0.0])
    for t in range(n):
        # predict
        sp=F@state; Pp=F@P@F.T+Q
        K=Pp@H.T / ((H@Pp@H.T)[0,0]+R)
        state=sp+K.flatten()*(z[t]- (H@sp)[0])
        P=(np.eye(2)-K@H)@Pp
        s[t]=state[0]
    return s
# ----------------------------------------------------------------------

def plot_live_hysteresis(
    instrument     = 1,
    prices_file    = "prices.txt",
    hma_period     = 4,
    # Kalman parms
    R=0.075, Q_level=4e-3, Q_trend=1e-5,
    band_pct       = 0.002,       # 0.1 % hysteresis pocket
    T1             = 340,
    T2             = 450,
    divider_w      = 0.3,
    divider_a      = 0.45
):
    price  = load_prices(prices_file, instrument)
    n      = len(price)
    T2     = min(T2, n)

    h_ser  = hma(price, hma_period)
    k_ser  = kalman(price, R, Q_level, Q_trend)

    # ------------ live hysteresis sign -----------------------------------
    band   = band_pct * price          # adaptive ±band around zero
    pos    = np.zeros(n, dtype=int)
    state  = 1 if k_ser[0] > h_ser[0] else -1

    for t in range(n):
        diff = k_ser[t] - h_ser[t]
        if   diff >  band[t]:
            state =  1
        elif diff < -band[t]:
            state = -1
        # else keep previous
        pos[t] = state

    # ------------ slice & plot -------------------------------------------
    x   = np.arange(T1, T2)
    p,w = price[T1:T2], h_ser[T1:T2]
    k,w2 = k_ser[T1:T2], pos[T1:T2]

    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, p,               color='black', label='Raw')
    ax.plot(x, h_ser[T1:T2], '--', color='C1', label=f'HMA({hma_period})')
    ax.plot(x, k_ser[T1:T2], '-.', color='C2', label='Kalman')

    for xi in x:
        ax.axvline(xi, color='black', lw=divider_w, alpha=divider_a, zorder=0)

    y0, y1 = p.min(), p.max()
    run_s, cur = x[0], w2[0]
    for xi, si in zip(x, w2):
        if si != cur:
            ax.axvspan(run_s, xi, facecolor='green' if cur==1 else 'red',
                       alpha=0.22, edgecolor=None)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1, facecolor='green' if cur==1 else 'red',
               alpha=0.22, edgecolor=None)

    ax.set_title(f'Live hysteresis: long if Kalman-HMA>{band_pct*100:.2f}%\n'
                 f'(green = long, red = short, no 1-bar flips)')
    ax.set_xlabel("Time Step"); ax.set_ylabel("Price"); ax.legend()
    plt.tight_layout(); plt.show()


if __name__ == "__main__":
    plot_live_hysteresis()


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─────────────────────────── basic helpers ────────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values


def wma(series: np.ndarray, period: int) -> np.ndarray:
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out


def hma(series: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)


def kalman_trend_smoother(prices, R=1.0, Q_level=1e-3, Q_trend=1e-5):
    n = len(prices)
    x = np.array([prices[0], 0.0])
    P = np.eye(2)
    F = np.array([[1, 1], [0, 1]])
    H = np.array([[1, 0]])
    Q = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

# ─────────────────────── plot combining rule ─────────────────────────────
def plot_kalman_vs_hma(
    instrument   : int,
    prices_file  : str  = "prices.txt",
    hma_period   : int  = 8,
    R            : float = 0.05,
    Q_level      : float = 4e-3,
    Q_trend      : float = 1e-5,
    kal_drive    : bool  = False,   # True → short when Kalman > HMA
    T1           : int   = 300,
    T2           : int   = 400,
    divider_w    : float = 0.3,
    divider_a    : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # compute lines
    h_line = hma(prices, hma_period)
    k_line = kalman_trend_smoother(prices, R, Q_level, Q_trend)

    # signal: compare levels
    if kal_drive:
        # inverted logic
        sign = np.where(k_line > h_line, -1, 1)
    else:
        sign = np.where(k_line > h_line, 1, -1)

    # slice
    x  = np.arange(T1, T2)
    pw = prices[T1:T2]
    hw = h_line[T1:T2]
    kw = k_line[T1:T2]
    sw = sign[T1:T2]

    # plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, pw, color="black", label="Raw Price")
    ax.plot(x, hw, "--", color="C1", lw=2, label=f"HMA({hma_period})")
    ax.plot(x, kw, "-.", color="C2", lw=2, label="Kalman")

    # thin dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # contiguous shading
    y0, y1 = pw.min(), pw.max()
    start, cur = x[0], sw[0]
    for xi, si in zip(x, sw):
        if si != cur:
            ax.axvspan(start, xi,
                       facecolor="green" if cur == 1 else "red",
                       alpha=0.22, edgecolor=None)
            start, cur = xi, si
    ax.axvspan(start, x[-1] + 1,
               facecolor="green" if cur == 1 else "red",
               alpha=0.22, edgecolor=None)

    logic = "short if Kalman>HMA" if kal_drive else "long if Kalman>HMA"
    ax.set_title(
        f"Instr {instrument}: Kalman vs HMA agreement shading\n({logic})"
    )
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


# ───────── demo ─────────
if __name__ == "__main__":
    plot_kalman_vs_hma(
        instrument  = 1,
        prices_file = "prices.txt",
        hma_period  = 5,
        R           = 0.21,
        Q_level     = 4e-3,
        Q_trend     = 1e-10,
        kal_drive   = True,     # flip to True to invert long/short logic
        T1          = 340,
        T2          = 450,
    )


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ────────────────────────────  basic helpers  ────────────────────────────
def load_prices(prices_file: str, instrument: int) -> np.ndarray:
    """Auto-discover prices.txt and return the chosen column."""
    for folder in (Path.cwd(), *Path.cwd().parents):
        f = folder / prices_file
        if f.exists():
            df = pd.read_csv(f, sep=r"\s+", header=None)
            break
    else:
        raise FileNotFoundError(prices_file)
    if not (0 <= instrument < df.shape[1]):
        raise IndexError("bad instrument index")
    return df.iloc[:, instrument].values


def wma(series: np.ndarray, period: int) -> np.ndarray:
    """Causal weighted MA with weights 1…period."""
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out


def hma(series: np.ndarray, period: int) -> np.ndarray:
    """Hull MA, fully causal."""
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)


def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5):
    """Two-state (level + trend) causal Kalman filter."""
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        # predict
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        # update
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out


# ───────── buffered-HMA logic (trend length vs confirmation) ─────────────
def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    """
    Buffered ±1 series:
    • If current side has lasted ≥N bars, flip on first opposite bar.
    • Else need X consecutive opposite bars to flip.
    """
    out           = np.empty_like(raw)
    state         = raw[0]
    same_streak   = 1
    opp_counter   = 0
    out[0]        = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_counter  = 0
        else:
            opp_counter += 1
            if same_streak >= N or opp_counter >= X:
                state       = r
                same_streak = 1
                opp_counter = 0
            else:
                same_streak = 0
        out[t] = state
    return out


# ────────────────────────────  main plot  ────────────────────────────────
def plot_hma_kalman_agree(
    instrument     : int,
    prices_file    : str   = "prices.txt",
    # HMA
    hma_period     : int   = 8,
    N_trend        : int   = 6,
    X_confirm      : int   = 3,
    # Kalman
    R              : float = 0.075,
    Q_level        : float = 4e-3,
    Q_trend        : float = 1e-5,
    pct_thresh     : float = 0.001,
    # window
    T1             : int   = 340,
    T2             : int   = 800,
    divider_w      : float = 0.3,
    divider_a      : float = 0.45,
):
    prices = load_prices(prices_file, instrument)
    n      = len(prices)
    T2     = min(T2, n)

    # 1) Buffered-HMA signal
    h_series   = hma(prices, hma_period)
    h_grad     = np.zeros(n); h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw      = np.where(h_grad > 0, 1, -1)
    h_signal   = buffered_signals(h_raw, N_trend, X_confirm)

    # 2) Kalman signal, with neutral band on price change
    k_series   = kalman_trend_smoother(prices, R, Q_level, Q_trend)
    pct_change = np.zeros(n); pct_change[1:] = (prices[1:] - prices[:-1]) / prices[:-1]
    k_dir      = np.zeros(n, dtype=int)
    k_dir[1:]  = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_signal   = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # 3) Agreement, then “hold-last” persistence
    agree      = np.where((h_signal == k_signal) & (k_signal != 0), h_signal, 0)
    state      = agree.copy()
    for t in range(1, n):
        if state[t] == 0:          # disagreement → keep previous regime
            state[t] = state[t-1]

    # 4) Slice
    x   = np.arange(T1, T2)
    p   = prices[T1:T2]
    h   = h_series[T1:T2]
    k   = k_series[T1:T2]
    s   = state[T1:T2]

    # 5) Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x, p, label="Raw Price", color="black")
    ax.plot(x, h, "--", label=f"HMA buffered ({hma_period})", color="C1")
    ax.plot(x, k, "-.", label="Kalman", color="C2")

    # vertical dividers
    for xi in x:
        ax.axvline(xi, color="black", lw=divider_w, alpha=divider_a, zorder=0)

    # shade by contiguous state
    y0, y1 = p.min(), p.max()
    run_s, cur = x[0], s[0]
    for xi, si in zip(x, s):
        if si != cur:
            ax.axvspan(run_s, xi,
                       facecolor="green" if cur == 1 else "red",
                       alpha=0.22, edgecolor=None)
            run_s, cur = xi, si
    ax.axvspan(run_s, x[-1]+1,
               facecolor="green" if cur == 1 else "red",
               alpha=0.22, edgecolor=None)

    ax.set_title(
        f"Instr {instrument}: persistent regime when models disagree\n"
        f"HMA N={N_trend}, X={X_confirm} | Kalman neutral band ±{pct_thresh*100:.2f}%"
    )
    ax.set_xlabel("Time Step"); ax.set_ylabel("Price")
    ax.legend(loc="lower left")
    plt.tight_layout(); plt.show()


# ──────────────────────────  demo call  ───────────────────────────────────
if __name__ == "__main__":
    plot_hma_kalman_agree(
        instrument  = 18,
        prices_file = "prices.txt",
        hma_period  = 4,
        N_trend     = 5,
        X_confirm   = 1,
        R           = 0.075,
        Q_level     = 4e-3,
        Q_trend     = 1e-5,
        pct_thresh  = 0.001,
        T1          = 240,
        T2          = 440,
    )


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# ---------- helpers reused from user's code ----------
def wma(series: np.ndarray, period: int) -> np.ndarray:
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)

def kalman_trend_smoother(prices, R=0.05, Q_level=4e-3, Q_trend=1e-5):
    n = len(prices)
    x = np.array([prices[0], 0.0])
    P = np.eye(2)
    F = np.array([[1, 1], [0, 1]])
    H = np.array([[1, 0]])
    Q = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

def buffered_signals(raw: np.ndarray, N: int, X: int) -> np.ndarray:
    out = np.empty_like(raw)
    state, same_streak, opp_counter = raw[0], 1, 0
    out[0] = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same_streak += 1
            opp_counter = 0
        else:
            opp_counter += 1
            if same_streak >= N or opp_counter >= X:
                state, same_streak, opp_counter = r, 1, 0
            else:
                same_streak = 0
        out[t] = state
    return out

# ---------- load prices ----------
prices_path = None
for folder in (Path.cwd(), *Path.cwd().parents):
    f = folder / "prices.txt"
    if f.exists():
        prices_path = f
        break
if prices_path is None:
    raise FileNotFoundError("prices.txt not found")

df_prices = pd.read_csv(prices_path, sep=r"\s+", header=None)
num_inst = min(50, df_prices.shape[1])
T_MAX = 801  # use 0..800 inclusive
prices_mat = df_prices.iloc[:T_MAX, :num_inst].values  # shape (T, N)

# ---------- parameters ----------
hma_period = 4
N_trend = 5
X_confirm = 1
R = 0.075
Q_level = 4e-3
Q_trend = 1e-5
pct_thresh = 0.001  # neutral band

trade_returns = []

for inst in range(num_inst):
    price_series = prices_mat[:, inst]
    # compute buffered HMA signal
    h_series = hma(price_series, hma_period)
    h_grad = np.zeros_like(h_series)
    h_grad[1:] = (h_series[1:] - h_series[:-1]) / h_series[:-1]
    h_raw = np.where(h_grad > 0, 1, -1)
    h_sig = buffered_signals(h_raw, N_trend, X_confirm)

    # kalman
    k_series = kalman_trend_smoother(price_series, R, Q_level, Q_trend)
    pct_change = np.zeros_like(price_series)
    pct_change[1:] = (price_series[1:] - price_series[:-1]) / price_series[:-1]
    k_dir = np.zeros_like(price_series, dtype=int)
    k_dir[1:] = np.where(k_series[1:] > k_series[:-1], 1, -1)
    k_sig = np.where(np.abs(pct_change) < pct_thresh, 0, k_dir)

    # agreement, hold last
    agree = np.where((h_sig == k_sig) & (k_sig != 0), h_sig, 0)
    state = agree.copy()
    for t in range(1, len(state)):
        if state[t] == 0:
            state[t] = state[t-1]

    # simulate trades
    pos = 0  # 1 long, -1 short, 0 flat
    entry_price = None
    for t in range(len(state)):
        sig = state[t]
        if sig != pos:
            # close existing
            if pos != 0 and entry_price is not None:
                exit_price = price_series[t]
                ret = (exit_price - entry_price) / entry_price if pos == 1 else (entry_price - exit_price) / entry_price
                trade_returns.append(ret)
            # open new if sig !=0
            if sig != 0:
                entry_price = price_series[t]
            else:
                entry_price = None
            pos = sig

# ---------- metrics ----------
trade_returns = np.array(trade_returns)
num_trades = len(trade_returns)
wins = trade_returns[trade_returns > 0]
win_rate = len(wins) / num_trades if num_trades else 0.0

# bins
bins = [-np.inf, -0.1, -0.02, 0, 0.02, 0.1, np.inf]
labels = ["<-10%", "-10% to -2%", "-2% to 0", "0 to 2%", "2% to 10%", ">10%"]
hist_counts = pd.cut(trade_returns, bins=bins, labels=labels).value_counts().sort_index()

print(f"Total trades: {num_trades}")
print(f"Win rate: {win_rate:.2%}")
print("\nDistribution of trade outcomes:")
print(hist_counts)



In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# ───────── helpers ──────────────────────────────────────────────────────────
def wma(series: np.ndarray, period: int) -> np.ndarray:
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)

def kalman_smoother(prices, R=0.075, Q_level=4e-3, Q_trend=1e-5):
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

def buffered_signals(raw, N, X):
    out, state = np.empty_like(raw), raw[0]
    same, opp = 1, 0
    out[0] = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0          # still counting choppy zone
        out[t] = state
    return out

# ───────── load price matrix (first 50 instruments, t=0..800) ─────────────
prices_file = "prices.txt"
for fold in (Path.cwd(), *Path.cwd().parents):
    path = fold / prices_file
    if path.exists():
        df = pd.read_csv(path, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

num_inst = min(50, df.shape[1])
T_MAX    = 801
matrix   = df.iloc[:T_MAX, :num_inst].values   # (T,N)

# ───────── parameters ─────────────────────────────────────────────────────
hma_period  = 4
N_trend     = 5
X_confirm   = 1
R           = 0.075
Q_level     = 4e-3
Q_trend     = 1e-5
pct_thresh  = 0.001       # Kalman neutral band

trade_returns = []

# ───────── iterate instruments ────────────────────────────────────────────
for inst in range(num_inst):
    price = matrix[:, inst]

    # buffered-HMA signal
    h = hma(price, hma_period)
    h_grad = np.zeros_like(h)
    h_grad[1:] = (h[1:] - h[:-1]) / h[:-1]
    h_raw  = np.where(h_grad > 0, 1, -1)
    h_sig  = buffered_signals(h_raw, N_trend, X_confirm)

    # Kalman signal with neutral band
    k = kalman_smoother(price, R, Q_level, Q_trend)
    pct = np.zeros_like(price)
    pct[1:] = (price[1:] - price[:-1]) / price[:-1]
    k_dir = np.zeros_like(price, dtype=int)
    k_dir[1:] = np.where(k[1:] > k[:-1], 1, -1)
    k_sig = np.where(np.abs(pct) < pct_thresh, 0, k_dir)

    # agreement + hold-last persistence
    agree = np.where((h_sig == k_sig) & (k_sig != 0), h_sig, 0)
    state = agree.copy()
    for t in range(1, len(state)):
        if state[t] == 0:
            state[t] = state[t-1]

    # ── simulate trades with **inverted** logic
    pos, entry_price = 0, None          # 1 = long, -1 = short, 0 = flat
    for t, sig in enumerate(state):
        inv_sig = -sig                  # ← invert prediction
        if inv_sig != pos:              # change position
            if pos != 0 and entry_price is not None:
                exit_px = price[t]
                ret = (exit_px - entry_price) / entry_price if pos == 1 \
                      else (entry_price - exit_px) / entry_price
                trade_returns.append(ret)
            # open new position
            if inv_sig != 0:
                entry_price = price[t]
            else:
                entry_price = None
            pos = inv_sig

# ───────── metrics ────────────────────────────────────────────────────────
trade_returns = np.array(trade_returns)
num_trades    = len(trade_returns)
wins          = trade_returns[trade_returns > 0]
win_rate      = len(wins) / num_trades if num_trades else 0.0

bins   = [-np.inf, -0.1, -0.02, 0, 0.02, 0.1, np.inf]
labels = ["<-10%", "-10% to -2%", "-2% to 0",
          "0 to 2%", "2% to 10%", ">10%"]
hist   = pd.cut(trade_returns, bins=bins, labels=labels).value_counts().sort_index()

print(f"Total trades: {num_trades}")
print(f"Win rate   : {win_rate:.2%}\n")
print("Distribution of trade outcomes:")
print(hist)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# ───────── helpers ──────────────────────────────────────────────────────────
def wma(series: np.ndarray, period: int) -> np.ndarray:
    n, w = len(series), np.arange(1, period + 1)
    out  = np.full(n, np.nan)
    S    = w.sum()
    for i in range(period - 1, n):
        out[i] = (w * series[i - period + 1 : i + 1]).sum() / S
    out[: period - 1] = series[: period - 1]
    return out

def hma(series: np.ndarray, period: int) -> np.ndarray:
    half, sqrtp = max(1, period // 2), max(1, int(np.sqrt(period)))
    return wma(2 * wma(series, half) - wma(series, period), sqrtp)

def kalman_smoother(prices, R=0.075, Q_level=4e-3, Q_trend=1e-5):
    n   = len(prices)
    x   = np.array([prices[0], 0.0])
    P   = np.eye(2)
    F   = np.array([[1, 1], [0, 1]])
    H   = np.array([[1, 0]])
    Q   = np.diag([Q_level, Q_trend])
    out = np.zeros(n)
    for t in range(n):
        x_prior = F @ x
        P_prior = F @ P @ F.T + Q
        y = prices[t] - (H @ x_prior)[0]
        S = (H @ P_prior @ H.T)[0, 0] + R
        K = (P_prior @ H.T) / S
        x = x_prior + (K.flatten() * y)
        P = (np.eye(2) - K @ H) @ P_prior
        out[t] = x[0]
    return out

def buffered_signals(raw, N, X):
    out, state = np.empty_like(raw), raw[0]
    same, opp = 1, 0
    out[0] = state
    for t in range(1, len(raw)):
        r = raw[t]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0          # still counting choppy zone
        out[t] = state
    return out

# ───────── load price matrix (first 50 instruments, t=0..800) ─────────────
prices_file = "prices.txt"
for fold in (Path.cwd(), *Path.cwd().parents):
    path = fold / prices_file
    if path.exists():
        df = pd.read_csv(path, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

num_inst = min(50, df.shape[1])
T_MAX    = 801
matrix   = df.iloc[:T_MAX, :num_inst].values   # (T,N)

# ───────── parameters ─────────────────────────────────────────────────────
hma_period  = 8
N_trend     = 6
X_confirm   = 2
R           = 0.05
Q_level     = 4e-3
Q_trend     = 5e-5
pct_thresh  = 0.002      # Kalman neutral band

trade_returns = []

# ───────── iterate instruments ────────────────────────────────────────────
for inst in range(num_inst):
    price = matrix[:, inst]

    # buffered-HMA signal
    h = hma(price, hma_period)
    h_grad = np.zeros_like(h)
    h_grad[1:] = (h[1:] - h[:-1]) / h[:-1]
    h_raw  = np.where(h_grad > 0, 1, -1)
    h_sig  = buffered_signals(h_raw, N_trend, X_confirm)

    # Kalman signal with neutral band
    k = kalman_smoother(price, R, Q_level, Q_trend)
    pct = np.zeros_like(price)
    pct[1:] = (price[1:] - price[:-1]) / price[:-1]
    k_dir = np.zeros_like(price, dtype=int)
    k_dir[1:] = np.where(k[1:] > k[:-1], 1, -1)
    k_sig = np.where(np.abs(pct) < pct_thresh, 0, k_dir)

    # agreement + hold-last persistence
    agree = np.where((h_sig == k_sig) & (k_sig != 0), h_sig, 0)
    state = agree.copy()
    for t in range(1, len(state)):
        if state[t] == 0:
            state[t] = state[t-1]

    # ── simulate trades with **inverted** logic
    pos, entry_price = 0, None          # 1 = long, -1 = short, 0 = flat
    for t, sig in enumerate(state):
        inv_sig = -sig                  # ← invert prediction
        if inv_sig != pos:              # change position
            if pos != 0 and entry_price is not None:
                exit_px = price[t]
                ret = (exit_px - entry_price) / entry_price if pos == 1 \
                      else (entry_price - exit_px) / entry_price
                trade_returns.append(ret)
            # open new position
            if inv_sig != 0:
                entry_price = price[t]
            else:
                entry_price = None
            pos = inv_sig

# ───────── metrics ────────────────────────────────────────────────────────
trade_returns = np.array(trade_returns)
num_trades    = len(trade_returns)
wins          = trade_returns[trade_returns > 0]
win_rate      = len(wins) / num_trades if num_trades else 0.0

bins   = [-np.inf, -0.1, -0.02, 0, 0.02, 0.1, np.inf]
labels = ["<-10%", "-10% to -2%", "-2% to 0",
          "0 to 2%", "2% to 10%", ">10%"]
hist   = pd.cut(trade_returns, bins=bins, labels=labels).value_counts().sort_index()

print(f"Total trades: {num_trades}")
print(f"Win rate   : {win_rate:.2%}\n")
print("Distribution of trade outcomes:")
print(hist)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import itertools, sys, time

# ───────────────────────────── helpers ────────────────────────────────────────
def wma(x, n):
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n),
               max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F@ s
        P = F@P@F.T + Q
        # update
        y = x[t] - (H@s)[0]
        S = (H@P@H.T)[0,0] + R
        K = (P@H.T)/S
        s = s + K.flatten()*y
        P = (np.eye(2) - K@H)@P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1,len(raw)):
        r = raw[i]
        if r==state:
            same +=1; opp = 0
        else:
            opp +=1
            if same>=N or opp>=X:
                state, same, opp = r,1,0
            else:
                same = 0
        out[i] = state
    return out

# ───────────────────────────── load prices ──────────────────────────────────
for f in (Path.cwd(), *Path.cwd().parents):
    p = f/"prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

T_MAX, N_MAX = 801, 50
prices = df.iloc[:T_MAX, :N_MAX].values
num_inst = prices.shape[1]

# ─────────────────────────── parameter grid ─────────────────────────────────
grid = dict(
    hma_period  = [4,6,8],
    N_trend     = [4,6],
    X_confirm   = [1,2],
    R           = [0.05,0.075],
    Q_level     = [2e-3,4e-3],
    Q_trend     = [1e-5,5e-5],
    pct_thresh  = [0.001,0.002],
)
param_sets = list(itertools.product(*grid.values()))
total_sets = len(param_sets)

# ─────────────────────────── run backtest ────────────────────────────────────
results = []
t_start = time.time()
for idx, vals in enumerate(param_sets,1):
    # break if >3 minutes
    if time.time() - t_start > 180:
        break

    hp, Nt, Xc, R, Ql, Qt, pct = vals
    all_returns = []
    total_preds, correct_preds = 0, 0

    for inst in range(num_inst):
        px = prices[:,inst]

        # HMA branch
        h = hma(px, hp)
        hg = np.zeros_like(h); hg[1:] = (h[1:] - h[:-1]) / h[:-1]
        h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)

        # Kalman branch
        k = kalman(px, R, Ql, Qt)
        chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1]) / px[:-1]
        k_dir = np.zeros_like(px,dtype=int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
        k_sig = np.where(np.abs(chg)<pct, 0, k_dir)

        # agreement + persist‐last
        agree = np.where((h_sig==k_sig)&(k_sig!=0), h_sig, 0)
        state = agree.copy()
        for t in range(1,len(state)):
            if state[t]==0:
                state[t]=state[t-1]

        # simulate inverted trades
        pos, entry = 0, None
        for t in range(len(state)):
            inv = -state[t]
            # for accuracy: compare inv vs next bar direction
            if t < len(state)-1 and inv!=0:
                true_dir = 1 if px[t+1]>px[t] else -1
                total_preds +=1
                if inv==true_dir:
                    correct_preds +=1

            if inv != pos:
                if pos!=0 and entry is not None:
                    exit_p = px[t]
                    r = ((exit_p-entry)/entry) if pos==1 else ((entry-exit_p)/entry)
                    all_returns.append(r)
                entry = px[t] if inv!=0 else None
                pos = inv

    # compute metrics
    tr = np.array(all_returns)
    win_rt = (tr>0).mean() if tr.size else 0
    avg_r  = tr.mean()     if tr.size else 0
    accuracy = correct_preds/total_preds if total_preds else 0

    results.append((accuracy, win_rt, avg_r, vals))

    # progress
    sys.stdout.write(
        f"\r[{idx}/{total_sets}] "
        f"hp={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
        f"pct={pct:.3f}  |  acc={accuracy:.2%} win={win_rt:.2%}"
    )
    sys.stdout.flush()

elapsed = time.time() - t_start
print(f"\nDone in {elapsed/60:.1f} min  using {len(results)} combos.")

# ─────────────────────────── sort & summarize ───────────────────────────────

# … [all helper definitions, grid‐search loop, etc. unchanged] …

# ─────────────────────────── sort & summarize ───────────────────────────────
# sort by average return (index 2 of each tuple), descending
results.sort(key=lambda x: x[2], reverse=True)
best   = results[0]
median = results[len(results)//2]
worst  = results[-1]

def fmt(res):
    acc, wr, avg, vals = res
    hp, Nt, Xc, R, Ql, Qt, pct = vals
    return (f" avg={avg:+.3%}, win={wr:.2%}, acc={acc:.2%}  │ "
            f"hp={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},pct={pct:.3f}")

print("\n\n=== SUMMARY (sorted by avg return) ===")
print("Best   :", fmt(best))
print("Median :", fmt(median))
print("Worst  :", fmt(worst))

# ────────────────── P&L buckets for BEST config ─────────────────────────────
_,_,_,(hp, Nt, Xc, R, Ql, Qt, pct) = best
trade_rtn = []
for inst in range(num_inst):
    px = prices[:,inst]
    # … re‐generate state exactly as before …
    h = hma(px,hp)
    hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1])/h[:-1]
    h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)
    k = kalman(px,R,Ql,Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1])/px[:-1]
    k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
    k_sig = np.where(np.abs(chg)<pct,0,k_dir)
    agree = np.where((h_sig==k_sig)&(k_sig!=0), h_sig, 0)
    state = agree.copy()
    for t in range(1,len(state)):
        if state[t]==0: state[t]=state[t-1]
    pos, entry = 0, None
    for t in range(len(state)):
        inv = -state[t]
        if inv!=pos:
            if pos!=0 and entry is not None:
                e = px[t]
                trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
            pos = inv
            entry = px[t] if inv!=0 else None

bins   = [-np.inf, -0.02, -0.01, 0, 0.01, 0.02, np.inf]
labels = ["<-2%", "-2%…-1%", "-1%…0", "0…1%", "1%…2%", ">2%"]
hist = pd.cut(np.array(trade_rtn), bins=bins, labels=labels).value_counts().sort_index()

print("\nBest‐config P&L buckets:")
print(hist)



In [ ]:
#Incorporated RSI, MACD, Volatility blindly into the algorithm and gridsearching paramters

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import itertools, sys, time

# ─────────────────────────────── helper functions ─────────────────────────────────

def wma(x, n):
    """Causal weighted MA with weights 1…n."""
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    """Hull MA, fully causal."""
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n), max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    """Two‐state causal Kalman smoother (level+trend)."""
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F @ s
        P = F @ P @ F.T + Q
        # update
        y = x[t] - (H@s)[0]
        S = (H@P@H.T)[0,0] + R
        K = (P@H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K@H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    """Buffered ±1 regime: if current side ≥N flip immediately, else require X confirms."""
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1,len(raw)):
        r = raw[i]
        if r==state:
            same += 1; opp = 0
        else:
            opp += 1
            if same>=N or opp>=X:
                state, same, opp = r,1,0
            else:
                same = 0
        out[i] = state
    return out

def rsi(series, period=14):
    """Classic RSI (Wilder smoothing)."""
    s = pd.Series(series)
    delta = s.diff().fillna(0)
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0,np.nan)
    r = 100 - (100/(1+rs))
    return r.fillna(50).values

def macd(series, fast=12, slow=26, sig=9):
    """MACD histogram = fast−slow EMA minus signal EMA."""
    s = pd.Series(series)
    ef = s.ewm(span=fast, adjust=False).mean()
    es = s.ewm(span=slow, adjust=False).mean()
    macd_line = ef - es
    sig_line = macd_line.ewm(span=sig, adjust=False).mean()
    return (macd_line - sig_line).values

def vol_at(series, window=20):
    """Causal rolling population‐stdDev."""
    return pd.Series(series).rolling(window, min_periods=1).std(ddof=0).values

# ───────────────────────────── load prices ───────────────────────────────────────
for f in (Path.cwd(), *Path.cwd().parents):
    p = f/"prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

T_MAX, N_MAX = 801, 50
prices = df.iloc[:T_MAX, :N_MAX].values
num_inst = prices.shape[1]

# ───────────────────────────── parameter grid ─────────────────────────────────────
grid = dict(
    hma_period  = [4,6],              # 2
    N_trend     = [4,6],              # 2
    X_confirm   = [1,2],              # 2
    R           = [0.05,0.075],       # 2
    Q_level     = [2e-3,4e-3],        # 2
    Q_trend     = [1e-5,5e-5],        # 2
    pct_thresh  = [0.001,0.002],      # 2
    vol_thresh  = [0.5,1.0],          # 2
)
param_sets = list(itertools.product(*grid.values()))
total = len(param_sets)             # 2^8 = 256

# RSI & MACD fixed params
RSI_PERIOD = 14; RSI_MID = 50
MACD_FAST, MACD_SLOW, MACD_SIG = 12,26,9
VOL_WINDOW = 20

# ───────────────────────────── run backtest ─────────────────────────────────────
results = []
t0 = time.time()
for idx, vals in enumerate(param_sets, 1):
    (hp, Nt, Xc, R, Ql, Qt, pct, vt) = vals
    trade_rtn = []

    for inst in range(num_inst):
        px = prices[:,inst]

        # HMA branch
        h = hma(px, hp)
        hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1]) / h[:-1]
        h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)

        # Kalman branch
        k = kalman(px, R, Ql, Qt)
        chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1]) / px[:-1]
        k_dir = np.zeros_like(px, dtype=int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
        k_sig = np.where(np.abs(chg)<pct, 0, k_dir)

        # RSI branch
        r = rsi(px, RSI_PERIOD)
        r_sig = np.where(r>RSI_MID,1,-1)

        # MACD branch
        m = macd(px, MACD_FAST, MACD_SLOW, MACD_SIG)
        m_sig = np.where(m>0,1,-1)

        # vol filtering
        v = vol_at(px, VOL_WINDOW)

        # agreement & persist‐last
        agree = np.where(
            (h_sig==k_sig)&(k_sig!=0)&
            (h_sig==r_sig)&(h_sig==m_sig),
            h_sig, 0
        )
        # vol threshold
        agree = np.where(v<vt, 0, agree)
        state = agree.copy()
        for t in range(1,len(state)):
            if state[t]==0:
                state[t]=state[t-1]

        # simulate inverted
        pos=None; entry=None
        for t,sig in enumerate(state):
            inv = -sig
            if pos is None:
                pos=inv; entry=px[t] if inv!=0 else None
            elif inv!=pos:
                if pos!=0 and entry is not None:
                    e=px[t]
                    trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
                pos=inv; entry=px[t] if inv!=0 else None

    tr = np.array(trade_rtn)
    avg = tr.mean() if tr.size else 0.0
    wr  = (tr>0).mean() if tr.size else 0.0
    results.append((avg, wr, vals))

    # progress
    sys.stdout.write(
        f"\r[{idx}/{total}] "
        f"hp={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
        f"pct={pct:.3f},vt={vt:.1f} → avg={avg:+.3%}"
    )
    sys.stdout.flush()

print(f"\nFinished in {(time.time()-t0)/60:.1f} min.")

# ────────────────────────── summarize by average P/L ───────────────────────────
results.sort(key=lambda x:x[0], reverse=True)
best, mid, worst = results[0], results[len(results)//2], results[-1]

def fmt(r):
    avg,wr,vals = r
    (hp,Nt,Xc,R,Ql,Qt,pct,vt) = vals
    return (f"avg={avg:+.3%}, win={wr:.2%} │ "
            f"h={hp},Nt={Nt},X={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
            f"pct={pct:.3f},vt={vt:.1f}")

print("\n\n=== SUMMARY (by avg P/L) ===")
print("BEST   :", fmt(best))
print("MEDIAN :", fmt(mid))
print("WORST  :", fmt(worst))

# ───── distribution of P&L for BEST config ────────────────────────────────────
best_vals = best[2]
trade_rtn = []
for inst in range(num_inst):
    px = prices[:,inst]
    # … (same logic as above) …
    h = hma(px,best_vals[0])
    h_grad = np.zeros_like(h); h_grad[1:]=(h[1:]-h[:-1])/h[:-1]
    h_sig = buffered(np.where(h_grad>0,1,-1), best_vals[1], best_vals[2])
    k = kalman(px, best_vals[3], best_vals[4], best_vals[5])
    chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1])/px[:-1]
    k_dir = np.zeros_like(px,dtype=int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
    k_sig = np.where(np.abs(chg)<best_vals[6],0,k_dir)
    r_sig = np.where(rsi(px,RSI_PERIOD)>RSI_MID,1,-1)
    m_sig = np.where(macd(px,MACD_FAST,MACD_SLOW,MACD_SIG)>0,1,-1)
    v = vol_at(px,VOL_WINDOW)
    agree = np.where((h_sig==k_sig)&(k_sig!=0)&(h_sig==r_sig)&(h_sig==m_sig),h_sig,0)
    agree = np.where(v<best_vals[7],0,agree)
    state = agree.copy()
    for t in range(1,len(state)):
        if state[t]==0: state[t]=state[t-1]
    pos=None; entry=None
    for t,sig in enumerate(state):
        inv=-sig
        if pos is None:
            pos=inv; entry=px[t] if inv!=0 else None
        elif inv!=pos:
            if pos!=0 and entry is not None:
                e=px[t]
                trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
            pos=inv; entry=px[t] if inv!=0 else None

dist = pd.cut(np.array(trade_rtn),
              bins=[-np.inf,-0.02,-0.01,0,0.01,0.02,np.inf],
              labels=["<-2%","-2%…-1%","-1%…0","0…1%","1%…2%",">2%"])
print("\nBest‐config trade P&L buckets:")
print(dist.value_counts().sort_index())


In [ ]:
#Gridsearching for best parameters identical tuning, utilizing RSI, MACD, and Volatility as additional filters but 
# for a smaller training window, general approach testing if parameters hold the same. If not we could be overfitting

#FIRST

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import itertools, sys, time

# ─────────────────────────────── helper functions ─────────────────────────────────

def wma(x, n):
    """Causal weighted MA with weights 1…n."""
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    """Hull MA, fully causal."""
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n),
               max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    """Two-state causal Kalman smoother (level + trend)."""
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F @ s
        P = F @ P @ F.T + Q
        # update
        y = x[t] - (H@s)[0]
        S = (H@P@H.T)[0,0] + R
        K = (P@H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K@H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    """Buffered ±1 regime: if current side ≥N flip immediately, else require X confirms."""
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1,len(raw)):
        r = raw[i]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r,1,0
            else:
                same = 0
        out[i] = state
    return out

def rsi(series, period=14):
    """Classic RSI (Wilder smoothing)."""
    s = pd.Series(series)
    delta = s.diff().fillna(0)
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0,np.nan)
    r  = 100 - (100/(1+rs))
    return r.fillna(50).values

def macd(series, fast=12, slow=26, sig=9):
    """MACD histogram = fast EMA – slow EMA minus signal EMA."""
    s = pd.Series(series)
    ef = s.ewm(span=fast, adjust=False).mean()
    es = s.ewm(span=slow, adjust=False).mean()
    macd_line = ef - es
    sig_line  = macd_line.ewm(span=sig, adjust=False).mean()
    return (macd_line - sig_line).values

def vol_at(series, window=20):
    """Causal rolling population‐stdDev."""
    return pd.Series(series).rolling(window, min_periods=1).std(ddof=0).values

# ────────────────────────────── load prices ───────────────────────────────────────
for f in (Path.cwd(), *Path.cwd().parents):
    p = f/"prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

# ───── restrict to first 500 timesteps and first 50 instruments ───────────────
T_MAX, N_MAX = 500, 50
prices = df.iloc[:T_MAX, :N_MAX].values
num_inst = prices.shape[1]

# ─────────────────────────── parameter grid ─────────────────────────────────────
grid = dict(
    hma_period  = [4,6],               # ↔ was [4,6]
    N_trend     = [4,6],
    X_confirm   = [1,2],
    R           = [0.05,0.075],
    Q_level     = [2e-3,4e-3],
    Q_trend     = [1e-5,5e-5],
    pct_thresh  = [0.001,0.002],
    vol_thresh  = [0.5,1.0],
)
param_sets = list(itertools.product(*grid.values()))
total = len(param_sets)  # 256 combos

# RSI & MACD fixed
RSI_PERIOD = 14; RSI_MID = 50
MACD_FAST, MACD_SLOW, MACD_SIG = 12,26,9
VOL_WINDOW = 20

# ───────────────────────────── run backtest ─────────────────────────────────────
results = []
t0 = time.time()
for idx, vals in enumerate(param_sets, 1):
    hp, Nt, Xc, R, Ql, Qt, pct, vt = vals
    trade_rtn = []

    for inst in range(num_inst):
        px = prices[:,inst]

        # HMA branch
        h = hma(px, hp)
        hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1]) / h[:-1]
        h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)

        # Kalman branch
        k = kalman(px, R, Ql, Qt)
        chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1])/px[:-1]
        k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
        k_sig = np.where(np.abs(chg)<pct, 0, k_dir)

        # RSI branch
        r = rsi(px, RSI_PERIOD)
        r_sig = np.where(r>RSI_MID, 1, -1)

        # MACD branch
        m = macd(px, MACD_FAST, MACD_SLOW, MACD_SIG)
        m_sig = np.where(m>0,1,-1)

        # volatility filter
        v = vol_at(px, VOL_WINDOW)

        # agreement & persisting
        agree = np.where(
            (h_sig==k_sig)&(k_sig!=0)&
            (h_sig==r_sig)&(h_sig==m_sig),
            h_sig, 0
        )
        agree = np.where(v<vt, 0, agree)
        state = agree.copy()
        for t in range(1,len(state)):
            if state[t]==0:
                state[t]=state[t-1]

        # simulate inverted entries/exits
        pos, entry = None, None
        for t,sig in enumerate(state):
            inv = -sig
            if pos is None:
                pos = inv
                entry = px[t] if inv!=0 else None
            elif inv != pos:
                if pos != 0 and entry is not None:
                    e = px[t]
                    trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
                pos = inv
                entry = px[t] if inv!=0 else None

    tr = np.array(trade_rtn)
    avg = tr.mean() if tr.size else 0.0
    wr  = (tr>0).mean() if tr.size else 0.0
    results.append((avg, wr, vals))

    sys.stdout.write(
        f"\r[{idx}/{total}] "
        f"hp={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
        f"pct={pct:.3f},vt={vt:.1f} → avg={avg:+.3%}"
    )
    sys.stdout.flush()

print(f"\nFinished in {(time.time()-t0)/60:.1f} min.\n")

# ───────────────────────── summary by average P/L ─────────────────────────────
results.sort(key=lambda x: x[0], reverse=True)
best, mid, worst = results[0], results[len(results)//2], results[-1]

def fmt(r):
    avg,wr,vals = r
    hp,Nt,Xc,R,Ql,Qt,pct,vt = vals
    return (f"avg={avg:+.3%}, win={wr:.2%}  │ "
            f"h={hp},Nt={Nt},X={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
            f"pct={pct:.3f},vt={vt:.1f}")

print("=== SUMMARY (by avg P/L) ===")
print("BEST   :", fmt(best))
print("MEDIAN :", fmt(mid))
print("WORST  :", fmt(worst))

# ────────────────── P&L buckets for BEST config ─────────────────────────────
_,_,(hp,Nt,Xc,R,Ql,Qt,pct,vt) = best
trade_rtn = []
for inst in range(num_inst):
    px = prices[:,inst]
    h = hma(px,hp)
    hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1])/h[:-1]
    h_sig = buffered(np.where(hg>0,1,-1),Nt,Xc)
    k = kalman(px,R,Ql,Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1])/px[:-1]
    k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
    k_sig = np.where(np.abs(chg)<pct,0,k_dir)
    r_sig = np.where(rsi(px,14)>50,1,-1)
    m_sig = np.where(macd(px,12,26,9)>0,1,-1)
    v = vol_at(px,20)
    agree = np.where((h_sig==k_sig)&(k_sig!=0)&(h_sig==r_sig)&(h_sig==m_sig),h_sig,0)
    agree = np.where(v<vt,0,agree)
    state = agree.copy()
    for t in range(1,len(state)):
        if state[t]==0: state[t]=state[t-1]
    pos, entry = None, None
    for t,sig in enumerate(state):
        inv = -sig
        if pos is None:
            pos=inv; entry=px[t] if inv!=0 else None
        elif inv!=pos:
            if pos!=0 and entry is not None:
                e=px[t]
                trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
            pos=inv; entry=px[t] if inv!=0 else None

dist = pd.cut(np.array(trade_rtn),
              bins=[-np.inf,-0.02,-0.01,0,0.01,0.02,np.inf],
              labels=["<-2%","-2%…-1%","-1%…0","0…1%","1%…2%",">2%"])
print("\nBest-config trade P&L buckets:")
print(dist.value_counts().sort_index())


In [ ]:
#Dealt with a slight cheating issue where the Average is filled slightly pre-emptively, buys on same bars close as it analyse instead of watining for t + 1

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import itertools, sys, time

# ─────────────────────────────── helper functions ─────────────────────────────────

def wma(x, n):
    """Causal weighted MA with weights 1…n."""
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    """Hull MA, fully causal."""
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n),
               max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    """Two‐state causal Kalman smoother (level + trend)."""
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F @ s
        P = F @ P @ F.T + Q
        # update
        y = x[t] - (H@s)[0]
        S = (H@P@H.T)[0,0] + R
        K = (P@H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K@H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    """Buffered ±1 regime: if current side ≥N flip immediately, else require X confirms."""
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1,len(raw)):
        r = raw[i]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0
        out[i] = state
    return out

def rsi(series, period=14):
    """Classic RSI (Wilder smoothing)."""
    s = pd.Series(series)
    delta = s.diff().fillna(0)
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0,np.nan)
    r  = 100 - (100/(1+rs))
    return r.fillna(50).values

def macd(series, fast=12, slow=26, sig=9):
    """MACD histogram = fast EMA – slow EMA minus signal EMA."""
    s = pd.Series(series)
    ef = s.ewm(span=fast, adjust=False).mean()
    es = s.ewm(span=slow, adjust=False).mean()
    macd_line = ef - es
    sig_line  = macd_line.ewm(span=sig, adjust=False).mean()
    return (macd_line - sig_line).values

def vol_at(series, window=20):
    """Causal rolling population‐stdDev."""
    return pd.Series(series).rolling(window, min_periods=1).std(ddof=0).values

# ───────────────────────────── load prices ───────────────────────────────────────
for f in (Path.cwd(), *Path.cwd().parents):
    p = f/"prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

T_MAX, N_MAX = 801, 50
prices = df.iloc[:T_MAX, :N_MAX].values
num_inst = prices.shape[1]

# ───────────────────────────── parameter grid ─────────────────────────────────────
grid = dict(
    hma_period  = [4,6],               # 2
    N_trend     = [4,6],               # 2
    X_confirm   = [1,2],               # 2
    R           = [0.05,0.075],        # 2
    Q_level     = [2e-3,4e-3],         # 2
    Q_trend     = [1e-5,5e-5],         # 2
    pct_thresh  = [0.001,0.002],       # 2
    vol_thresh  = [0.5,1.0],           # 2
)
param_sets = list(itertools.product(*grid.values()))
total = len(param_sets)               # 256 combos

# RSI & MACD fixed params
RSI_PERIOD = 14; RSI_MID = 50
MACD_FAST, MACD_SLOW, MACD_SIG = 12,26,9
VOL_WINDOW = 20

# ───────────────────────────── run backtest ─────────────────────────────────────
results = []
t0 = time.time()
for idx, vals in enumerate(param_sets, 1):
    hp, Nt, Xc, R, Ql, Qt, pct, vt = vals
    trade_rtn = []

    for inst in range(num_inst):
        px = prices[:,inst]

        # 1) HMA branch
        h = hma(px, hp)
        hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1]) / h[:-1]
        h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)

        # 2) Kalman branch
        k = kalman(px, R, Ql, Qt)
        chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1]) / px[:-1]
        k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
        k_sig = np.where(np.abs(chg)<pct, 0, k_dir)

        # 3) RSI branch
        r = rsi(px, RSI_PERIOD)
        r_sig = np.where(r>RSI_MID,1,-1)

        # 4) MACD branch
        m = macd(px, MACD_FAST, MACD_SLOW, MACD_SIG)
        m_sig = np.where(m>0,1,-1)

        # 5) volatility filter
        v = vol_at(px, VOL_WINDOW)

        # 6) agreement & hold-last
        agree = np.where(
            (h_sig==k_sig)&(k_sig!=0)&
            (h_sig==r_sig)&(h_sig==m_sig),
            h_sig, 0
        )
        agree = np.where(v<vt, 0, agree)
        state = agree.copy()
        for t in range(1,len(state)):
            if state[t]==0:
                state[t]=state[t-1]

        # 7) simulate with *next-bar* execution
        pos, entry = None, None
        # we can only trade up to t = len(state)-2 to have a next bar
        for t in range(len(state)-1):
            inv = -state[t]
            if pos is None:
                pos   = inv
                entry = px[t+1] if inv!=0 else None
            elif inv != pos:
                # close existing at px[t+1]
                if pos!=0 and entry is not None:
                    exit_px = px[t+1]
                    trade_rtn.append(
                        (exit_px-entry)/entry if pos==1
                        else (entry-exit_px)/entry
                    )
                # open new if inv!=0
                pos   = inv
                entry = px[t+1] if inv!=0 else None

    tr = np.array(trade_rtn)
    avg = tr.mean() if tr.size else 0.0
    wr  = (tr>0).mean() if tr.size else 0.0
    results.append((avg, wr, vals))

    sys.stdout.write(
        f"\r[{idx}/{total}] "
        f"hp={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
        f"pct={pct:.3f},vt={vt:.1f} → avg={avg:+.3%}"
    )
    sys.stdout.flush()

print(f"\nFinished in {(time.time()-t0)/60:.1f} min.\n")

# ───────────────────────── summary by average P/L ─────────────────────────────
results.sort(key=lambda x: x[0], reverse=True)
best, mid, worst = results[0], results[len(results)//2], results[-1]

def fmt(r):
    avg,wr,vals = r
    hp,Nt,Xc,R,Ql,Qt,pct,vt = vals
    return (f"avg={avg:+.3%}, win={wr:.2%} │ "
            f"h={hp},Nt={Nt},X={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
            f"pct={pct:.3f},vt={vt:.1f}")

print("=== SUMMARY (by avg P/L) ===")
print("BEST   :", fmt(best))
print("MEDIAN :", fmt(mid))
print("WORST  :", fmt(worst))

# ────────────────── P&L buckets for BEST config ─────────────────────────────
_,_,(hp,Nt,Xc,R,Ql,Qt,pct,vt) = best
trade_rtn = []
for inst in range(num_inst):
    px = prices[:,inst]
    h = hma(px,hp)
    hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1])/h[:-1]
    h_sig = buffered(np.where(hg>0,1,-1),Nt,Xc)
    k = kalman(px,R,Ql,Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1])/px[:-1]
    k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
    k_sig = np.where(np.abs(chg)<pct,0,k_dir)
    r_sig = np.where(rsi(px,RSI_PERIOD)>RSI_MID,1,-1)
    m_sig = np.where(macd(px,MACD_FAST,MACD_SLOW,MACD_SIG)>0,1,-1)
    v = vol_at(px,VOL_WINDOW)
    agree = np.where((h_sig==k_sig)&(k_sig!=0)&(h_sig==r_sig)&(h_sig==m_sig),h_sig,0)
    agree = np.where(v<vt,0,agree)
    state = agree.copy()
    for t in range(1,len(state)):
        if state[t]==0: state[t]=state[t-1]
    pos,entry=None,None
    for t in range(len(state)-1):
        inv = -state[t]
        if pos is None:
            pos=inv; entry=px[t+1] if inv!=0 else None
        elif inv!=pos:
            if pos!=0 and entry is not None:
                e=px[t+1]
                trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
            pos=inv; entry=px[t+1] if inv!=0 else None

dist = pd.cut(np.array(trade_rtn),
              bins=[-np.inf,-0.02,-0.01,0,0.01,0.02,np.inf],
              labels=["<-2%","-2%…-1%","-1%…0","0…1%","1%…2%",">2%"])
print("\nBest‐config trade P&L buckets:")
print(dist.value_counts().sort_index())


In [ ]:
#Including Grisearch on RSI, MACD, and Volatility as additional filters for boosted performance, this is first 8000 timesteos without any cheating issues ( I think ) 

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import itertools, sys, time

# ─────────────────────────────── helper functions ─────────────────────────────────

def wma(x, n):
    """Causal weighted MA with weights 1…n."""
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    """Hull MA, fully causal."""
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n),
               max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    """Two‐state causal Kalman smoother (level + trend)."""
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F @ s
        P = F @ P @ F.T + Q
        # update
        y = x[t] - (H@s)[0]
        S = (H@P@H.T)[0,0] + R
        K = (P@H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K@H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    """Buffered ±1 regime: if current side ≥N flip immediately, else require X confirms."""
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1,len(raw)):
        r = raw[i]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0
        out[i] = state
    return out

def rsi(series, period):
    """Classic RSI (Wilder smoothing)."""
    s = pd.Series(series)
    delta = s.diff().fillna(0)
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0,np.nan)
    r  = 100 - (100/(1+rs))
    return r.fillna(50).values

def macd(series, fast, slow, sig):
    """MACD histogram = fast EMA − slow EMA minus signal."""
    s = pd.Series(series)
    ef = s.ewm(span=fast, adjust=False).mean()
    es = s.ewm(span=slow, adjust=False).mean()
    mline = ef - es
    sigl  = mline.ewm(span=sig, adjust=False).mean()
    return (mline - sigl).values

def vol_at(series, window):
    """Causal rolling population‐stdDev."""
    return pd.Series(series).rolling(window, min_periods=1).std(ddof=0).values

# ────────────────────────────── load prices ───────────────────────────────────────
for f in (Path.cwd(), *Path.cwd().parents):
    p = f/"prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r"\s+", header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

# restrict to first 800 timesteps and first 50 instruments
T_MAX, N_MAX = 800, 50
prices = df.iloc[:T_MAX, :N_MAX].values
num_inst = prices.shape[1]

# ───────────────────────────── parameter grid ─────────────────────────────────────
grid = dict(
    hma_period  = [4,8],               # 2 choices
    N_trend     = [4],                 # 1
    X_confirm   = [1,2],               # 2
    R           = [0.05,0.075],        # 2
    Q_level     = [2e-3],              # 1
    Q_trend     = [1e-5],              # 1
    pct_thresh  = [0.001,0.002],       # 2
    vol_thresh  = [0.5,1.0],           # 2
    RSI_PERIOD  = [14,21],             # 2
    RSI_MID     = [50,55],             # 2
    MACD_FAST   = [12,15],             # 2
    MACD_SLOW   = [26,30],             # 2
    MACD_SIG    = [9],                 # 1
)
param_sets = list(itertools.product(*grid.values()))
total = len(param_sets)  # 2*1*2*2*1*1*2*2*2*2*2*1 = 512

# ───────────────────────────── run backtest ─────────────────────────────────────
results = []
t0 = time.time()
for idx, vals in enumerate(param_sets, 1):
    hp, Nt, Xc, R, Ql, Qt, pct, vt, \
    rpi, rmid, mf, ms, msig = vals

    trade_rtn = []
    for inst in range(num_inst):
        px = prices[:,inst]

        # HMA signal
        h = hma(px, hp)
        hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1]) / h[:-1]
        h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)

        # Kalman signal
        k = kalman(px, R, Ql, Qt)
        chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1]) / px[:-1]
        k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
        k_sig = np.where(np.abs(chg)<pct, 0, k_dir)

        # RSI signal
        r = rsi(px, rpi)
        r_sig = np.where(r>rmid, 1, -1)

        # MACD signal
        m = macd(px, mf, ms, msig)
        m_sig = np.where(m>0, 1, -1)

        # Vol filter
        v = vol_at(px, 20)

        # agreement & hold-last
        agree = np.where(
            (h_sig==k_sig)&(k_sig!=0)&
            (h_sig==r_sig)&(h_sig==m_sig),
            h_sig, 0
        )
        agree = np.where(v<vt, 0, agree)
        state = agree.copy()
        for t in range(1,len(state)):
            if state[t]==0:
                state[t]=state[t-1]

        # simulate trades on next-bar execution
        pos, entry = None, None
        for t in range(len(state)-1):
            inv = -state[t]
            if pos is None:
                pos   = inv
                entry = px[t+1] if inv!=0 else None
            elif inv != pos:
                if pos!=0 and entry is not None:
                    exit_px = px[t+1]
                    trade_rtn.append(
                        (exit_px-entry)/entry if pos==1
                        else (entry-exit_px)/entry
                    )
                pos   = inv
                entry = px[t+1] if inv!=0 else None

    tr = np.array(trade_rtn)
    avg = tr.mean() if tr.size else 0.0
    wr  = (tr>0).mean() if tr.size else 0.0
    results.append((avg, wr, vals))

    sys.stdout.write(
        f"\r[{idx}/{total}] "
        f"hp={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
        f"pct={pct:.3f},vt={vt:.1f},rpi={rpi},rmid={rmid},"
        f"mf={mf},ms={ms},msig={msig} → avg={avg:+.3%}"
    )
    sys.stdout.flush()

print(f"\nFinished in {(time.time()-t0)/60:.1f} min.\n")

# ───────────────────────── summary by average P/L ─────────────────────────────
results.sort(key=lambda x: x[0], reverse=True)
best, mid, worst = results[0], results[len(results)//2], results[-1]

def fmt(r):
    avg, wr, vals = r
    hp, Nt, Xc, R, Ql, Qt, pct, vt, \
    rpi, rmid, mf, ms, msig = vals
    return (f"avg={avg:+.3%}, win={wr:.2%} │ "
            f"h={hp},Nt={Nt},Xc={Xc},R={R:.3f},Ql={Ql:.1e},Qt={Qt:.1e},"
            f"pct={pct:.3f},vt={vt:.1f},rpi={rpi},rmid={rmid},"
            f"mf={mf},ms={ms},msig={msig}")

print("=== SUMMARY (by avg P/L) ===")
print("BEST   :", fmt(best))
print("MEDIAN :", fmt(mid))
print("WORST  :", fmt(worst))

# ────────────────── P&L buckets for BEST config ─────────────────────────────
_,_,best_vals = best
(hp, Nt, Xc, R, Ql, Qt, pct, vt,
 rpi, rmid, mf, ms, msig) = best_vals

trade_rtn = []
for inst in range(num_inst):
    px = prices[:,inst]
    # re-generate state exactly as above...
    h = hma(px,hp)
    hg = np.zeros_like(h); hg[1:] = (h[1:]-h[:-1])/h[:-1]
    h_sig = buffered(np.where(hg>0,1,-1),Nt,Xc)
    k = kalman(px,R,Ql,Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1])/px[:-1]
    k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
    k_sig = np.where(np.abs(chg)<pct,0,k_dir)
    r = rsi(px,rpi);     r_sig = np.where(r>rmid,1,-1)
    m = macd(px,mf,ms,msig); m_sig = np.where(m>0,1,-1)
    v = vol_at(px,20)
    agree = np.where((h_sig==k_sig)&(k_sig!=0)&(h_sig==r_sig)&(h_sig==m_sig),h_sig,0)
    agree = np.where(v<vt,0,agree)
    state = agree.copy()
    for t in range(1,len(state)):
        if state[t]==0: state[t]=state[t-1]
    pos,entry=None,None
    for t in range(len(state)-1):
        inv = -state[t]
        if pos is None:
            pos=inv; entry=px[t+1] if inv!=0 else None
        elif inv!=pos:
            if pos!=0 and entry is not None:
                e=px[t+1]
                trade_rtn.append((e-entry)/entry if pos==1 else (entry-e)/entry)
            pos=inv; entry=px[t+1] if inv!=0 else None

dist = pd.cut(
    np.array(trade_rtn),
    bins=[-np.inf,-0.02,-0.01,0,0.01,0.02,np.inf],
    labels=["<-2%","-2%…-1%","-1%…0","0…1%","1%…2%",">2%"]
)
print("\nBest‐config trade P&L buckets:")
print(dist.value_counts().sort_index())


In [ ]:
#Plotting on optimal 800, full paramter usage and printing between T1 and T2 for instrument X

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys

# ─── Helper functions ────────────────────────────────────────────────────────

def wma(x, n):
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n),
               max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F @ s
        P = F @ P @ F.T + Q
        # update
        y = x[t] - (H @ s)[0]
        S = (H @ P @ H.T)[0,0] + R
        K = (P @ H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K @ H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1, len(raw)):
        r = raw[i]
        if r == state:
            same += 1
            opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0
        out[i] = state
    return out

def rsi(series, period):
    s = pd.Series(series)
    delta = s.diff().fillna(0)
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    r  = 100 - (100/(1+rs))
    return r.fillna(50).values

def macd(series, fast, slow, sig):
    s = pd.Series(series)
    ef = s.ewm(span=fast, adjust=False).mean()
    es = s.ewm(span=slow, adjust=False).mean()
    mline = ef - es
    sigl  = mline.ewm(span=sig, adjust=False).mean()
    return (mline - sigl).values

def vol_at(series, window):
    return pd.Series(series).rolling(window, min_periods=1).std(ddof=0).values

# ─── Load price matrix ────────────────────────────────────────────────────────
for folder in (Path.cwd(), *Path.cwd().parents):
    path = folder / "prices.txt"
    if path.exists():
        df = pd.read_csv(path, sep=r'\s+', header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

# restrict to first 800 timesteps and first 50 instruments
T_MAX, N_MAX = 800, 50
prices = df.iloc[:T_MAX, :N_MAX].values

# ─── Optimized hyperparameters ───────────────────────────────────────────────
hp, Nt, Xc = 4, 4, 1
R, Ql, Qt = 0.075, 2e-3, 1e-5
pct, vt = 0.002, 1.0
rpi, rmid = 21, 55
mf, ms, msig = 12, 30, 9

# ─── Backtest simulation ─────────────────────────────────────────────────────
trade_rtn = []
num_inst = prices.shape[1]

for inst in range(num_inst):
    px = prices[:, inst]
    
    # HMA signal
    h = hma(px, hp)
    hg = np.zeros_like(h); hg[1:] = (h[1:] - h[:-1]) / h[:-1]
    h_sig = buffered(np.where(hg > 0, 1, -1), Nt, Xc)
    
    # Kalman signal
    k = kalman(px, R, Ql, Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:] - px[:-1]) / px[:-1]
    k_dir = np.zeros_like(px, int); k_dir[1:] = np.where(k[1:] > k[:-1], 1, -1)
    k_sig = np.where(np.abs(chg) < pct, 0, k_dir)
    
    # RSI signal
    r = rsi(px, rpi)
    r_sig = np.where(r > rmid, 1, -1)
    
    # MACD signal
    m = macd(px, mf, ms, msig)
    m_sig = np.where(m > 0, 1, -1)
    
    # Volatility filter
    v = vol_at(px, 20)
    
    # Agreement & persistence
    agree = np.where(
        (h_sig == k_sig) & (k_sig != 0) &
        (h_sig == r_sig) & (h_sig == m_sig),
        h_sig, 0
    )
    agree = np.where(v < vt, 0, agree)
    state = agree.copy()
    for t in range(1, len(state)):
        if state[t] == 0:
            state[t] = state[t-1]
    
    # Simulate trades (next-bar execution)
    pos = None
    entry = None
    for t in range(len(state)-1):
        inv = -state[t]
        if pos is None:
            pos = inv
            entry = px[t+1] if inv != 0 else None
        elif inv != pos:
            if pos != 0 and entry is not None:
                exit_px = px[t+1]
                trade_rtn.append((exit_px - entry)/entry if pos == 1 else (entry - exit_px)/entry)
            pos = inv
            entry = px[t+1] if inv != 0 else None

trade_rtn = np.array(trade_rtn)
avg = trade_rtn.mean() * 100  # in percent
win_rate = (trade_rtn > 0).mean() * 100  # in percent

print(f"Average P/L: {avg:+.3f}%")
print(f"Win rate  : {win_rate:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# ─── Plotting function ────────────────────────────────────────────────────────
def plot_trades_for_instrument(prices, instrument, 
                               hp, Nt, Xc, R, Ql, Qt, pct, vt,
                               rpi, rmid, mf, ms, msig):
    """
    prices: 2D numpy array [T × N]
    instrument: integer index of column
    the rest: your optimized hyperparams
    """
    px = prices[:, instrument]
    T = len(px)
    
    # 1) reproduce the state series
    h = hma(px, hp)
    hg = np.zeros_like(h); hg[1:] = (h[1:] - h[:-1]) / h[:-1]
    h_sig = buffered(np.where(hg > 0, 1, -1), Nt, Xc)
    
    k = kalman(px, R, Ql, Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:] - px[:-1]) / px[:-1]
    k_dir = np.zeros_like(px, int); k_dir[1:] = np.where(k[1:] > k[:-1], 1, -1)
    k_sig = np.where(np.abs(chg) < pct, 0, k_dir)
    
    r = rsi(px, rpi); r_sig = np.where(r > rmid, 1, -1)
    m = macd(px, mf, ms, msig); m_sig = np.where(m > 0, 1, -1)
    v = vol_at(px, 20)
    
    agree = np.where(
        (h_sig==k_sig)&(k_sig!=0)&(h_sig==r_sig)&(h_sig==m_sig),
        h_sig, 0
    )
    agree = np.where(v < vt, 0, agree)
    state = agree.copy()
    for t in range(1, T):
        if state[t] == 0:
            state[t] = state[t-1]
    
    # 2) next-bar execution → pos[t] = -state[t-1]
    pos = np.zeros(T, int)
    for t in range(1, T):
        pos[t] = -state[t-1]
    
    # 3) plot
    x = np.arange(T)
    ymin, ymax = px.min(), px.max()
    
    fig, ax = plt.subplots(figsize=(12,5))
    ax.plot(x, px, color="black", lw=1.5, label="Price")
    ax.fill_between(x, ymin, ymax, where=(pos==1),
                    facecolor="green", alpha=0.3, step="pre", label="Long")
    ax.fill_between(x, ymin, ymax, where=(pos==-1),
                    facecolor="red",   alpha=0.3, step="pre", label="Short")
    ax.set_title(f"Instrument {instrument}: Price & Trades")
    ax.set_xlabel("Time step")
    ax.set_ylabel("Price")
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


# ─── Usage ───────────────────────────────────────────────────────────────────
# (assuming you’ve already loaded `prices` and set the hyperparams)
plot_trades_for_instrument(
    prices, 
    instrument=7,
    hp=4, Nt=4, Xc=1,
    R=0.075, Ql=2e-3, Qt=1e-5,
    pct=0.002, vt=1.0,
    rpi=21, rmid=55,
    mf=12, ms=30, msig=9
)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# ─── Core helpers ────────────────────────────────────────────────────────────

def wma(x, n):
    n = int(n)
    w = np.arange(1, n+1)
    S = w.sum()
    out = np.full(len(x), np.nan)
    for i in range(n-1, len(x)):
        out[i] = (w * x[i-n+1:i+1]).sum() / S
    out[:n-1] = x[:n-1]
    return out

def hma(x, n):
    n = int(n)
    return wma(2*wma(x, max(1,n//2)) - wma(x,n),
               max(1,int(np.sqrt(n))))

def kalman(x, R, Ql, Qt):
    n = len(x)
    F = np.array([[1,1],[0,1]])
    H = np.array([[1,0]])
    Q = np.diag([Ql, Qt])
    s = np.array([x[0],0.0])
    P = np.eye(2)
    out = np.zeros(n)
    for t in range(n):
        # predict
        s = F @ s
        P = F @ P @ F.T + Q
        # update
        y = x[t] - (H @ s)[0]
        S = (H @ P @ H.T)[0,0] + R
        K = (P @ H.T) / S
        s = s + K.flatten()*y
        P = (np.eye(2) - K @ H) @ P
        out[t] = s[0]
    return out

def buffered(raw, N, X):
    out = np.empty_like(raw)
    state, same, opp = raw[0], 1, 0
    out[0] = state
    for i in range(1, len(raw)):
        r = raw[i]
        if r == state:
            same += 1; opp = 0
        else:
            opp += 1
            if same >= N or opp >= X:
                state, same, opp = r, 1, 0
            else:
                same = 0
        out[i] = state
    return out

def rsi(series, period):
    s = pd.Series(series)
    delta = s.diff().fillna(0)
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    r  = 100 - (100/(1+rs))
    return r.fillna(50).values

def macd(series, fast, slow, sig):
    s = pd.Series(series)
    ef = s.ewm(span=fast, adjust=False).mean()
    es = s.ewm(span=slow, adjust=False).mean()
    mline = ef - es
    sigl  = mline.ewm(span=sig, adjust=False).mean()
    return (mline - sigl).values

def vol_at(series, window):
    return pd.Series(series).rolling(window, min_periods=1).std(ddof=0).values

# ─── Load price data ─────────────────────────────────────────────────────────

for folder in (Path.cwd(), *Path.cwd().parents):
    p = folder / "prices.txt"
    if p.exists():
        df = pd.read_csv(p, sep=r'\s+', header=None)
        break
else:
    raise FileNotFoundError("prices.txt not found")

# take first 800 days × first 50 instruments:
prices = df.iloc[:800, :50].values

# ─── “Best” hyperparameters ─────────────────────────────────────────────────

hp, Nt, Xc = 4, 4, 1
R, Ql, Qt = 0.075, 2e-3, 1e-5
pct, vt   = 0.002, 1.0
rpi, rmid = 21, 55
mf, ms, msig = 12, 30, 9

# commission parameters
commission_rate = 0.0005  # 5 bps per side → ~10 bps round trip

# ─── Backtest + commission adjustment ────────────────────────────────────────

trade_returns = []
n_inst = prices.shape[1]

for inst in range(n_inst):
    px = prices[:, inst]
    T  = len(px)

    # generate the persistent regime state
    h   = hma(px, hp)
    hg  = np.zeros_like(h); hg[1:] = (h[1:] - h[:-1]) / h[:-1]
    h_sig = buffered(np.where(hg>0,1,-1), Nt, Xc)

    k   = kalman(px, R, Ql, Qt)
    chg = np.zeros_like(px); chg[1:] = (px[1:]-px[:-1]) / px[:-1]
    k_dir = np.zeros_like(px,int); k_dir[1:] = np.where(k[1:]>k[:-1],1,-1)
    k_sig = np.where(np.abs(chg)<pct, 0, k_dir)

    r   = rsi(px, rpi);   r_sig = np.where(r>rmid,1,-1)
    m   = macd(px, mf, ms, msig); m_sig = np.where(m>0,1,-1)
    v   = vol_at(px, 20)

    agree = np.where(
        (h_sig==k_sig)&(k_sig!=0)&
        (h_sig==r_sig)&(h_sig==m_sig),
        h_sig, 0
    )
    agree = np.where(v<vt, 0, agree)
    state = agree.copy()
    for t in range(1, T):
        if state[t]==0:
            state[t] = state[t-1]

    # simulate trades with next-bar fills & commission
    pos, entry = None, None
    for t in range(T-1):
        inv = -state[t]        # side we take at bar t+1
        if pos is None:
            pos   = inv
            entry = px[t+1] if inv!=0 else None
        elif inv != pos:
            if pos!=0 and entry is not None:
                exit_px = px[t+1]
                raw_ret = ((exit_px-entry)/entry) if pos==1 else ((entry-exit_px)/entry)
                # subtract round-trip commission (%):
                net_ret = raw_ret - 2*commission_rate
                trade_returns.append(net_ret)
            pos   = inv
            entry = px[t+1] if inv!=0 else None

# convert and summarize
trade_returns = np.array(trade_returns)
avg_net = trade_returns.mean() * 100  # %
print(f"Average net return (commission only): {avg_net:+.3f}%")
